In [3]:
import pandas.io.sql as sql
from lxml.html import parse
from urllib.request import urlopen
import pandas as pd
import urllib 
import numpy as np
from matplotlib.pyplot import plot,savefig 
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import statsmodels.api as sm
import scipy.stats as stats 

In [4]:
def get_sina_stock(stockid,start="1990-12-19",end=pd.datetime.now(),isstock=True,fuquan=True):    
    def _unpack(row, kind='td'):
        elts = row.findall('.//%s' % kind)
        return [val.text_content().strip() for val in elts] 
    def parse_options_data(table,isstock,fuquan):
        rows = table.findall('.//tr')
        data = [_unpack(r) for r in rows[2:]]
        if isstock & fuquan:
            colnames = ['date','open','high','close','low','vol','amount','ratio'] 
        else:
            colnames = ['date','open','high','close','low','vol','amount']
        data= pd.DataFrame(data,columns=colnames)
        data.index=pd.to_datetime(data["date"])
        data=data.drop("date",axis=1)
        data=data.ix[range(len(data)-1,-1,-1)]
        return  data
    data=pd.DataFrame()
    daterange=pd.date_range(start,end,freq="Q")
    daterange=daterange.insert(len(daterange),daterange[-1]+1)
    for cq in daterange: 
        if isstock:
            if fuquan:
                url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_FuQuanMarketHistory/stockid/'+str(stockid)+'.phtml?year='+str(cq.year)+'&jidu='+str(cq.month/3)
            else:
                url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_MarketHistory/stockid/'+str(stockid)+'.phtml?year='+str(cq.year)+'&jidu='+str(cq.month/3)            
        else:            
            url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_MarketHistory/stockid/'+str(stockid)+'/type/S.phtml?year='+str(cq.year)+'&jidu='+str(cq.month/3)
        try:
            parsed = parse(urlopen(url))
        except:
            print ("download failed for year=" +str(cq.year)+',  jidu='+str(cq.month/3))
            continue
        doc = parsed.getroot()
        tables=doc.findall('.//table')     
        da= tables[-1] # last table 19 for fuquan data and 4 for index
        datatem = parse_options_data(da,isstock,fuquan)
        data=pd.concat([data,datatem])
    return pd.DataFrame(data[start:end],dtype=float)    

In [7]:
IDlist=['600615','600606','600621','600638']
data1=get_sina_stock('600615','1991-1-1','2016-11-16').close
data2=get_sina_stock('600606','1991-1-1','2016-11-16').close
data3=get_sina_stock('600621','1991-1-1','2016-11-16').close
data4=get_sina_stock('600638','1991-1-1','2016-11-16').close
data1.name='600615'#丰华股份
data2.name='600606'#绿地控股
data3.name='600621'#华鑫股份
data4.name='600638'#新黄浦
data=pd.concat([data1,data2,data3,data4],axis=1)

In [8]:
rtn_m=data.resample('M').last().pct_change().dropna()
rtn_m.describe()
rtn_m

,600615,600606,600621,600638
date,,,,
1993-04-30,0.457335,0.536670,0.672745,0.822535
1993-05-31,-0.252074,-0.217368,-0.333061,-0.337602
1993-06-30,0.089252,0.007519,0.001939,0.040000
1993-07-31,-0.066082,-0.050005,-0.112902,-0.060256
1993-08-31,0.008489,-0.015791,0.047479,0.249591
1993-09-30,0.046777,0.026741,0.064608,0.009935
1993-10-31,-0.043789,-0.015618,0.028989,-0.039809
1993-11-30,0.242990,0.103180,0.285208,0.180472
1993-12-31,-0.227070,-0.266192,-0.164380,-0.169428


In [ ]:
Emu = DataFrame()
Emu = rtn_m.mean()
Emu = Emu.transpose()
Emu.plot()
plt.title('Mean Monthly Return')
plt.xlabel('time')
plt.ylabel('return')

Esigma = DataFrame()
Esigma = rtn_m.std()
Esigma = Esigma.transpose()
Esigma.plot()
plt.title('Standard Deviation')
plt.xlabel('time')
plt.ylabel('SD')